<a href="https://colab.research.google.com/github/pythonkvs/seminars/blob/main/%D0%AD%D1%84%D1%84%D0%B5%D0%BA%D1%82%D0%B8%D0%B2%D0%BD%D0%BE%D1%81%D1%82%D1%8C_%D0%BA%D0%BE%D0%B4%D0%B0_%D0%BF%D0%BE%D1%82%D0%BE%D0%BA%D0%B8_%D0%BF%D1%80%D0%BE%D1%86%D0%B5%D1%81%D1%81%D1%8B/%D0%AD%D1%84%D1%84%D0%B5%D0%BA%D1%82%D0%B8%D0%B2%D0%BD%D0%BE%D1%81%D1%82%D1%8C_%D0%BA%D0%BE%D0%B4%D0%B0_21_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://compscicenter.ru/courses/python/2015-autumn/classes/1561/

# Класс `Matrix`

In [2]:
import random

class Matrix(list):
    @classmethod
    def zeros(cls, shape):
        n_rows, n_cols = shape
        return cls([[0] * n_cols for i in range(n_rows)])

    @classmethod
    def random(cls, shape):
        M, (n_rows, n_cols) = cls(), shape
        for i in range(n_rows):
            M.append([random.randint(-255, 255)
                      for j in range(n_cols)])
        return M

    def transpose(self):
        n_rows, n_cols = self.shape
        return self.__class__(zip(*self))

    @property
    def shape(self):
        return ((0, 0) if not self else
                (len(self), len(self[0])))

In [65]:
def matrix_product(X, Y):
    """Computes the matrix product of X and Y.

    >>> X = Matrix([[1], [2], [3]])
    >>> Y = Matrix([[4, 5, 6]])
    >>> matrix_product(X, Y)
    [[4, 5, 6], [8, 10, 12], [12, 15, 18]]
    >>> matrix_product(Y, X)
    [[32]]
    """
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    # верим, что с размерностями всё хорошо
    Z = Matrix.zeros((n_xrows, n_ycols))
    for i in range(n_xrows):
        for j in range(n_xcols):
            for k in range(n_ycols):
                Z[i][k] += X[i][j] * Y[j][k]
    return Z

In [66]:
%doctest_mode

Exception reporting mode: Plain
Doctest mode is: ON


In [67]:
>>> X = Matrix([[1], [2], [3]])
>>> Y = Matrix([[4, 5, 6]])
>>> matrix_product(X, Y)
[[4, 5, 6], [8, 10, 12], [12, 15, 18]]
>>> matrix_product(Y, X)
[[32]]

[[32]]

In [68]:
%doctest_mode

Exception reporting mode: Context
Doctest mode is: OFF


# Измерение времени работы

Кажется, всё работает, но насколько быстро? Воспользуемся "магической" командой `timeit`, чтобы проверить.

In [69]:
%%timeit shape = 64, 64; X = Matrix.random(shape); Y = Matrix.random(shape)
matrix_product(X, Y)

10 loops, best of 5: 110 ms per loop


Итого: умножение двух матриц размера 64x64 занимает 0.1 секунды.

Определим вспомогательную функцию `bench`, которая генерирует случайные матрицы указанного размера, а затем `n_iter` раз умножает их в цикле.

In [70]:
def bench(shape=(64, 64), n_iter=16):
    X = Matrix.random(shape)
    Y = Matrix.random(shape)
    for iter in range(n_iter):
        matrix_product(X, Y)    

Воспользуемся модулем `cProfile` для поиска проблемы.

In [46]:
import cProfile
source = open("faster_python_faster.py").read()
cProfile.run(source, sort="tottime")

         41393 function calls in 1.853 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       16    1.829    0.114    1.830    0.114 <string>:26(matrix_product)
     8192    0.007    0.000    0.015    0.000 random.py:174(randrange)
     8192    0.006    0.000    0.008    0.000 random.py:224(_randbelow)
     8192    0.003    0.000    0.018    0.000 random.py:218(randint)
      128    0.003    0.000    0.021    0.000 <string>:13(<listcomp>)
     8217    0.002    0.000    0.002    0.000 {method 'getrandbits' of '_random.Random' objects}
        1    0.001    0.001    1.852    1.852 <string>:47(bench)
        1    0.001    0.001    1.853    1.853 {built-in method builtins.exec}
     8192    0.001    0.000    0.001    0.000 {method 'bit_length' of 'int' objects}
       16    0.000    0.000    0.000    0.000 <string>:7(<listcomp>)
        2    0.000    0.000    0.021    0.011 <string>:9(random)
        1    0.000    0.000    1.85

Результат предсказуемый и довольно бесполезный: >90% времени работы происходит в функции `matrix_product`. Попробуем посмотреть на неё по внимательней с помощью модуля `line_profiler`.

In [47]:
pip install line_profiler

In [71]:
%load_ext line_profiler
%lprun -f matrix_product bench()

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


Заметим, что операция `list.__getitem__` не бесплатна. Переставим местами циклы `for` так, чтобы код делал меньше обращений по индексу.

In [72]:
def matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    for i in range(n_xrows):
        Xi = X[i]
        for k in range(n_ycols):
            acc = 0
            for j in range(n_xcols):
                acc += Xi[j] * Y[j][k]
            Z[i][k] = acc
    return Z

In [73]:
%lprun -f matrix_product bench()

На секунду быстрее, но всё равно слишком медленно: >30% времени уходит исключительно на итерацию! Поправим это.

In [74]:
def matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    for i in range(n_xrows):
        Xi, Zi = X[i], Z[i]
        for k in range(n_ycols):
            Zi[k] = sum(Xi[j] * Y[j][k] for j in range(n_xcols))
    return Z

In [75]:
%lprun -f matrix_product bench()

Функции `matrix_product` сильно похорошело. Но, кажется, это не предел. Попробуем снова убрать лишние обращения по индексу из самого внутреннего цикла.

In [5]:
def matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    Yt = Y.transpose()  # <--
    for i, (Xi, Zi) in enumerate(zip(X, Z)):
        for k, Ytk in enumerate(Yt):
            Zi[k] = sum(Xi[j] * Ytk[j] for j in range(n_xcols))
    return Z

## Измерение времени работы: резюме

* Измерить время работы функции можно с помощью
функции `timeit` из модуля `timeit`.
* Найти узкое место в программе — с помощью модуля
`cProfile`.
* Оценить вклад каждой строки кода в общее время работы
функции — с помощью модуля `line_profiler`.
* В IPython есть “магические” команды для всех трёх типов
измерений:
    * `%timeit`,
    * `%prun`,
    * `%lprun`.

## Сравнение с NumPy

* NumPy — библиотека для научных вычислений на Python.
* В основе библиотеки — ndarray — многомерный
типизированный массив.
* Сравним результаты наших стараний с ndarray:

In [3]:
shape = 64, 64
X, Y = Matrix.random(shape), Matrix.random(shape)

In [6]:
 %timeit -n100 matrix_product(X, Y)

100 loops, best of 5: 40.2 ms per loop


In [7]:
import numpy as np
X = np.random.randint(-255, 255, shape)
Y = np.random.randint(-255, 255, shape)

In [8]:
%timeit -n100 X.dot(Y)

100 loops, best of 5: 324 µs per loop


## AOT и JIT компиляция кода на Python

* Дальнейшие способы ускорения кода на Python
предполагают его преобразование в машинный код либо
до, либо в момент его исполнения.
* Ahead-of-time компиляция.
    * Python C-API: пишем код на С и реализуем к нему
интерфейс, понятный интерпретатору CPython.
    * Пишем код на подмножестве Python и преобразуем его в
код на C++ (Pythran) или C (Cython), использующий C-API
интепретатора CPython.
* Just-in-time компиляция: пишем код на Python и пытаемся
сделать его быстрее в момент исполнения.
    * PyPy: следим за исполнением программы и компилируем в
машинный код наиболее частые пути в ней.
    * Транслируем специальным образом помеченный код на
Python в LLVM (Numba) или C++ (HOPE), а затем
компилируем в машинный код.

# Numba

Numba не работает с встроенными списками. Перепишем функцию `matrix_product` с использованием ndarray.

In [77]:
import numba
import numpy as np


@numba.jit
def jit_matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

Посмотрим, что получилось.

In [78]:
shape = 64, 64
X = np.random.randint(-255, 255, shape)
Y = np.random.randint(-255, 255, shape)

%timeit -n100 jit_matrix_product(X, Y)

The slowest run took 11.20 times longer than the fastest. This could mean that an intermediate result is being cached.
100 loops, best of 5: 229 µs per loop


## Numba: резюме

* Numba — это JIT компилятор для Python кода, основанный
на трансляции в LLVM.
* В теории Numba не требует никаких изменений в коде,
кроме использования декоратора `numba.jit`.
* На практике далеко не любой код поддаётся эффективной
оптимизации.

# Cython

Cython — это:
* типизированное расширение языка Python,
* оптимизирующий компилятор Python и Cython в код на C,
использующий C-API интерпретатора CPython.

“Магическая” команда cython компилирует содержимое ячейки
с помощью Cython, а затем загружает все имена из
скомпилированного модуля в глобальное пространство имён.

In [10]:
%load_ext cython

In [11]:
%%cython
def f():
    return 42
def g():
    return []

In [12]:
f

<function _cython_magic_c1f94d1c4d0b98a0945d13fa04230b80.f>

In [13]:
g

<function _cython_magic_c1f94d1c4d0b98a0945d13fa04230b80.g>

In [14]:
f(), g()

(42, [])

In [80]:
%%cython -a
import random

class Matrix(list):
    @classmethod
    def zeros(cls, shape):
        n_rows, n_cols = shape
        return cls([[0] * n_cols for i in range(n_rows)])

    @classmethod
    def random(cls, shape):
        M, (n_rows, n_cols) = cls(), shape
        for i in range(n_rows):
            M.append([random.randint(-255, 255)
                      for j in range(n_cols)])
        return M

    def transpose(self):
        n_rows, n_cols = self.shape
        return self.__class__(zip(*self))

    @property
    def shape(self):
        return ((0, 0) if not self else
                (int(len(self)), int(len(self[0]))))

    
def cy_matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    Yt = Y.transpose()
    for i, Xi in enumerate(X):
        for k, Ytk in enumerate(Yt):
            Z[i][k] = sum(Xi[j] * Ytk[j] for j in range(n_xcols))
    return Z

* Чем интенсивнее цвет, тем менее специфичен тип
выражения и тем медленней выполняется фрагмент кода.
* Обилие желтого цвета намекает, что результат компиляции
функции `cy_matrix_product` мало чем отличается от её
версии на Python, потому что все объекты имеют тип
`PyObject`.

In [81]:
X = Matrix.random(shape)
Y = Matrix.random(shape)

In [82]:
%timeit -n100 cy_matrix_product(X, Y)

100 loops, best of 5: 24.3 ms per loop


Проблема в том, что Cython не может эффективно оптимизировать работу со списками, которые могут содержать элементы различных типов, поэтому перепишем `matrix_product` с использованием *ndarray*.

In [83]:
X = np.random.randint(-255, 255, size=shape)
Y = np.random.randint(-255, 255, size=shape)

In [84]:
%%cython -a
import numpy as np

def cy_matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [85]:
%timeit -n100 cy_matrix_product(X, Y)

100 loops, best of 5: 184 ms per loop


Как же так! Стало только хуже, причём большинство кода всё ещё использует вызовы Python. Избавимся от них, про аннотировав код типами.

In [86]:
%%cython -a
import numpy as np
cimport numpy as np

def cy_matrix_product(np.ndarray X, np.ndarray Y):
    cdef int n_xrows = X.shape[0]
    cdef int n_xcols = X.shape[1]
    cdef int n_yrows = Y.shape[0]
    cdef int n_ycols = Y.shape[1]
    cdef np.ndarray Z
    Z = np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [87]:
%timeit -n100 cy_matrix_product(X, Y)

100 loops, best of 5: 188 ms per loop


К сожалению, типовые аннотации не изменили время работы, потому что тело самого вложенного цикла Cython оптимизировать не смог. Fatality-time: укажем тип элементов в *ndarray*.

In [88]:
%%cython -a
import numpy as np
cimport numpy as np

def cy_matrix_product(np.ndarray[np.int64_t, ndim=2] X,
                      np.ndarray[np.int64_t, ndim=2] Y):
    cdef int n_xrows = X.shape[0]
    cdef int n_xcols = X.shape[1]
    cdef int n_yrows = Y.shape[0]
    cdef int n_ycols = Y.shape[1]
    cdef np.ndarray[np.int64_t, ndim=2] Z = \
        np.zeros((n_xrows, n_ycols), dtype=np.int64)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [89]:
%timeit -n100 cy_matrix_product(X, Y)

100 loops, best of 5: 803 µs per loop


Попробуем пойти дальше и отключить проверки на выход за границы массива и переполнение целочисленных типов.

In [90]:
%%cython -a
import numpy as np

cimport cython
cimport numpy as np

@cython.boundscheck(False)
@cython.overflowcheck(False)
def cy_matrix_product(np.ndarray[np.int64_t, ndim=2] X, 
                      np.ndarray[np.int64_t, ndim=2] Y):
    cdef int n_xrows = X.shape[0]
    cdef int n_xcols = X.shape[1]
    cdef int n_yrows = Y.shape[0]
    cdef int n_ycols = Y.shape[1]
    cdef np.ndarray[np.int64_t, ndim=2] Z = \
        np.zeros((n_xrows, n_ycols), dtype=np.int64)
    for i in range(n_xrows):        
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [91]:
%timeit -n100 cy_matrix_product(X, Y)

100 loops, best of 5: 464 µs per loop


Cython — удобный инструмент для написания критичного
по производительности кода на Python-подобном
синтаксисе.

https://github.com/MelLain/mipt-python/blob/spring-2021/lectures/08-efficiency.ipynb